# Spectral Analysis & Linear Algebra Utilities

Demonstrates spectral graph tools, matrix decompositions, and visualization
helpers from `esapp.utils`. The notebook covers vector field visualization,
path and cycle graph Laplacians, normalized Laplacian spectral analysis,
Takagi factorization for complex symmetric matrices, the Hermitify
transformation, custom colormaps, and physical constants.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from esapp.utils import (
    pathlap, pathincidence, normlap, eigmax, sorteig,
    hermitify, takagi, MU0,
    format_plot, plot_vecfield, darker_hsv_colormap,
)

In [ ]:
# Plotting functions (hidden from documentation)
import sys; sys.path.insert(0, "..")
from plot_helpers import (
    plot_vecfield_gallery, plot_graph_operators,
    plot_normlap_spectrum, plot_hermitify, plot_colormap_scales,
)

## 1. Vector Field Gallery

Several vector fields plotted with `plot_vecfield`, which color-codes arrows by angle.

In [ ]:
X, Y = np.meshgrid(np.linspace(-2, 2, 30), np.linspace(-2, 2, 30))
Xc, Yc = X - 0.0, Y - 0.0

fields = {
    'Source': (Xc, Yc),
    'Vortex': (-Yc, Xc),
    'Saddle': (Xc, -Yc),
    'Shear': (Yc, np.zeros_like(Yc)),
}

plot_vecfield_gallery(X, Y, fields)

## 2. Graph Laplacians: Path and Cycle

`pathlap` and `pathincidence` construct Laplacians and incidence matrices for path and cycle graphs.

In [ ]:
# Build path and cycle graph operators
N = 8
L_path = pathlap(N, periodic=False)
B_path = pathincidence(N, periodic=False)
L_cycle = pathlap(N, periodic=True)
B_cycle = pathincidence(N, periodic=True)

In [ ]:
plot_graph_operators(
    [L_path, L_cycle, B_path, B_cycle],
    ['Path Laplacian', 'Cycle Laplacian', 'Path Incidence', 'Cycle Incidence'],
    vranges=[(-2, 2), (-2, 2), (-1, 1), (-1, 1)],
    suptitle=f'Graph Operators (N={N})')

### Verify L = B @ B.T

For the **cycle** graph, `pathincidence` returns an N x N matrix (N edges), so `B @ B.T`
matches the Laplacian directly. For the **path** graph, only the first N-1 columns
represent real edges.

In [ ]:
# Cycle: B is N x N (N edges), so B @ B.T == L directly
L_cycle_check = B_cycle @ B_cycle.T
print("Cycle: L == B @ B.T:", np.allclose(L_cycle, L_cycle_check))

# Path: only first N-1 columns are real edges
B_path_trimmed = B_path[:, :N-1]
L_path_check = B_path_trimmed @ B_path_trimmed.T
print("Path:  L == B[:,:N-1] @ B[:,:N-1].T:", np.allclose(L_path, L_path_check))

Cycle: L == B @ B.T: True
Path:  L == B[:,:N-1] @ B[:,:N-1].T: False


## 3. Normalized Laplacian and Spectral Analysis

In [ ]:
L_norm, D, D_inv = normlap(L_cycle, return_scaling=True)
evals = np.linalg.eigvalsh(L_norm)

plot_normlap_spectrum(L_norm, evals)

print(f'Largest eigenvalue (eigmax): {eigmax(L_cycle):.4f}')

## 4. Takagi Factorization

Decomposes a complex symmetric matrix M = U * Sigma * U^T.

In [ ]:
# Create a complex symmetric matrix
np.random.seed(42)
A = np.random.randn(4, 4) + 1j * np.random.randn(4, 4)
M = A + A.T  # symmetrize (M = M^T, not M = M^H)

U, sigma = takagi(M)

print("Singular values:", np.round(sigma, 4))

# Verify: M = U @ diag(sigma) @ U.T
M_reconstructed = U @ np.diag(sigma) @ U.T
print(f"Reconstruction error: {np.linalg.norm(M - M_reconstructed):.2e}")

Singular values: [7.1369 5.0195 1.5215 0.8544]
Reconstruction error: 9.38e-15


## 5. Hermitify

Converts a complex symmetric matrix to Hermitian form.

In [ ]:
H = hermitify(M)

print('Original symmetric (M = M^T):', np.allclose(M, M.T))
print('Hermitified (H = H^H):       ', np.allclose(H, H.conj().T))

plot_hermitify(M, H)

## 6. Custom Colormaps

The `darker_hsv_colormap` creates a darker version of the HSV colormap, useful for vector field angle encoding.

In [ ]:
plot_colormap_scales([1.0, 0.7, 0.4])

## 7. Physical Constants

The `MU0` constant provides the permeability of free space, used in GIC electric field calculations.

In [ ]:
print(f"MU0 = {MU0:.6e} H/m")
print(f"Used in GIC: E = -MU0 * dH/dt")

MU0 = 1.256637e-06 H/m
Used in GIC: E = -MU0 * dH/dt
